In [1]:
from selenium import webdriver
from core import MySQLcompatible
from bs4 import BeautifulSoup
import requests
import re
import utils
import json
import datetime

insert into ignore

In [2]:
tabelas = {}

tabelas['jogos_uni'] = (
"CREATE TABLE `jogos_uni` ("
"  `id` int(11) NOT NULL,"
"  `titulo` varchar(250) NOT NULL,"
"  `data` datetime NOT NULL,"
"  `slugLiga` varchar(120) DEFAULT NULL,"
"  `pais` varchar(100) DEFAULT NULL,"
"  `liga` varchar(100) DEFAULT NULL,"
"  `status` int(11) NOT NULL,"
"  `posicao` int(11) NOT NULL"
") ENGINE=MyISAM DEFAULT CHARSET=latin1;")

tabelas['modal_uni'] = (
"CREATE TABLE IF NOT EXISTS `modal_uni` ("
"  `id` int(11) NOT NULL AUTOINCREMENT,"
"  `jogo_id` int(11) NOT NULL,"
"  `odd_id` int(11) NOT NULL,"
"  `cat_id` int(11) NOT NULL,"
"  `categoria` varchar(250) NOT NULL,"
"  `propriedade` varchar(250) NOT NULL,"
"  `valor` decimal(8,2) NOT NULL DEFAULT '0.00',"
"  `status` int(11) NOT NULL"
") ENGINE=MyISAM DEFAULT CHARSET=latin1;")

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome(options=options)
browser.get("https://bets93.net/")
browser.find_element_by_xpath("//div[@class='lateral']/div/ul").click() #clica no botao "todos"

In [4]:
pattern_campeonato = re.compile(r"c_visivel")
pattern_jogo = re.compile(r"j_visivel_")

In [5]:
soup = BeautifulSoup(browser.page_source, "html.parser")
browser.quit()

In [6]:
tabela_jogos = soup.find(class_="jogos")
jogos = tabela_jogos.findAll("div",recursive=False)

In [7]:
headers = {
    'sec-fetch-mode': 'cors',
    'cookie': '__cfduid=d286a6b04f21dc52264db8885f48b8f971570143846',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en,en-US;q=0.9,pt-BR;q=0.8,pt;q=0.7',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'referer': 'https://bets93.net/',
    'authority': 'bets93.net',
    'x-requested-with': 'XMLHttpRequest',
    'sec-fetch-site': 'same-origin',
}

In [35]:
id_jogos = []
for jogo in jogos:
    attr = jogo.get("id")
    if pattern_campeonato.match(attr):
        NOME_CAMP = jogo.find(class_="camp").text
        slugLiga = NOME_CAMP.lower().replace(' ','-')
        slugLiga = re.sub(pattern=r'(^-|-$)',repl='', string=slugLiga)
        pais = NOME_CAMP.split()[0]
        liga = NOME_CAMP.split()[1:]
        liga = ' '.join(liga)
        status = 1
        posicao = 1
    elif pattern_jogo.match(attr):
        id_jogo = attr.split('_')[-1]
        id_jogos.append(id_jogo)
        titulo = jogo.find(class_="times fundojogos").text.split()[:-3]
        titulo = ' '.join(titulo)        
        data_hora = jogo.find(class_="datahora").text
        
        ano = int(data_hora.split()[0].split('/')[2])
        mes = int(data_hora.split()[0].split('/')[1])
        dia = int(data_hora.split()[0].split('/')[0])
        horas = int(data_hora.split()[-1].split(':')[0])
        minutos = int(data_hora.split()[-1].split(':')[1])
        
        data_hora = str(datetime.datetime(ano,mes,dia,horas,minutos))
        
        params = (('id_jogo', id_jogo),)

        response = requests.get('https://bets93.net/api.php', headers=headers, params=params)        
        odd_id = []
        categoria = []
        propriedade = []
        valor = []
        
        tabela2 = []
        
        for dicionario in response.json():
            tabela2.append((dicionario['id_tipo_modalidade'], dicionario['id_modalidade'], dicionario['id_odd'], dicionario['odd']))
#             print((dicionario['id_tipo_modalidade'], dicionario['id_modalidade'], dicionario['id_odd'], dicionario['odd']))
#             for k,v in dicionario.items():
#                 if k == 'id_odd': odd_id.append(int(v))
#                 elif k == 'id_modalidade': categoria.append(int(v))
#                 elif k == 'id_tipo_modalidade': propriedade.append(int(v))
#                 elif k == 'odd': valor.append(float(v))
        
        #preciso ordenar os valores pela propriedade
        
#         print(f'tamanho de odd_id: {len(odd_id)}')
#         print(f'tamanho de propriedade: {len(propriedade)}')
#         print(f'tamanho de categoria: {len(categoria)}')
#         print(f'tamanho de valor: {len(valor)}')
        
#         print(sorted(odd_id))
#         print(sorted(propriedade))
#         print(sorted(categoria))
#         print(sorted(valor))
        
        
        tabela1 = {
            'id_jogo':id_jogo, 'titulo': titulo, 'data_hora': data_hora, 'slugLiga': slugLiga,
            'pais':pais, 'liga': liga, 'status': status, 'posicao': posicao
        }
#         tabela2 = {
#             'jogo_id': id_jogo, 'odd_id':odd_id, 'cat_id':1, 'categoria':categoria,
#             'propriedade':propriedade, 'valor':valor, 'id_modal':1, 'status':1
#         }
        break

In [52]:
sorted(tabela2, key=lambda x:int(x[0])/1)

[('1', '1', '13953', 1.887),
 ('1', '2', '13954', 3.825),
 ('1', '3', '13955', 4.284),
 ('2', '4', '13898', '1.22'),
 ('2', '7', '13899', '1.83'),
 ('2', '5', '13900', 1.2125),
 ('3', '218', '13896', 1.5066),
 ('3', '219', '13897', '2.2'),
 ('3', '258', '13986', 2.2),
 ('3', '259', '13987', '1.4'),
 ('3', '222', '13988', 3.2),
 ('3', '223', '13989', '1.22'),
 ('4', '12', '13880', 3.128),
 ('4', '385', '13881', '3.4'),
 ('4', '13', '13882', 6.65),
 ('4', '386', '13883', '8'),
 ('4', '14', '13884', '4.5'),
 ('4', '387', '13885', '13'),
 ('7', '37', '13949', '1.67'),
 ('7', '27', '13950', 1.98),
 ('7', '25', '14046', '13'),
 ('7', '26', '14047', '4.33'),
 ('7', '36', '14068', 1.089),
 ('7', '38', '14069', 2.2695),
 ('7', '39', '14070', '4.8'),
 ('8', '45', '13871', '2.88'),
 ('8', '46', '13872', '15'),
 ('8', '47', '13873', '34'),
 ('8', '48', '13874', '4.75'),
 ('8', '49', '13875', '6'),
 ('8', '50', '13876', '9'),
 ('8', '51', '13877', '21'),
 ('8', '52', '13878', '15'),
 ('8', '53', '1

In [22]:
print("\nTABELA 1")
print(json.dumps(tabela1, indent=4))


TABELA 1
{
    "id_jogo": "82916039",
    "titulo": "Hertha Berlin x Fortuna Dusseldorf",
    "data_hora": "2019-10-04 14:30:00",
    "slugLiga": "alemanha-bundesliga",
    "pais": "Alemanha",
    "liga": "Bundesliga",
    "status": 1,
    "posicao": 1
}
